# 04_0_ray_calculation
This notebook calculates the intersection of rays (for sunrays and daylight 'rays') with the context mesh from voxel centroid towards sun. This results in two dataframes, one that contains information on daylight availability and one that contains information on sunlight availability. 

For the daylight availability, the script imports a sunpath from ladybug. This Sunpath takes as input a longtitude and latitude over which to take the sun information.

For each voxel it casts a ray towards the sunpath. Because of the context arround the building, this ray could be blocked- the voxel does not receive light from this sunray.
For all the voxels that have received light, it then casts the ray from the voxel to the sun backwards, to check whether it blocks sun (casts a shadow) from the context. 


## 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
from ladybug.sunpath import Sunpath
from scipy.interpolate import RegularGridInterpolator
import digital_twinning as dt
import pandas as pd


## 1. Import Meshes (context + envelope)

### 1.1. Load Meshes

In [2]:
#loading envelope and context from obj 
envelope_path = os.path.relpath('../data/OBJ_files/building_envelope.obj')
context_path = os.path.relpath('../data/OBJ_files/final_BAG3D.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

True
False


### 1.2. Visualize Meshes (with pyvista)

In [3]:
# convert mesh to pv_mesh
# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(dt.tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(dt.tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(84701.87724865344, 448134.6587486534, 327.4372486534429),
 (84384.44, 447817.2215, 10.0),
 (0.0, 0.0, 1.0)]

## 2. Import Lattice (envelope)

### 2.1. Load the Envelope Lattice

In [4]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/CSV_files/voxelized_envelope.csv')

envelope_lattice = tg.lattice_from_csv(lattice_path)

#setting the full lattice 
full_lattice = envelope_lattice * 0 + 1

### 2.2. Visualize the Context Mesh + Envelope Lattice

In [5]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the context mesh: white
p.add_mesh(dt.tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(84702.22192807864, 448135.0034280786, 328.2819280786408),
 (84384.44, 447817.2215, 10.5),
 (0.0, 0.0, 1.0)]

## 3. Sun Vectors


### 3.1. Compute Sun Vectors

In [6]:
translation_point = envelope_mesh.center_mass
translation_point[2] = 0

sun_vectors = dt.sun_positions()

sun_vectors = sun_vectors * 300
sun_points = np.add(sun_vectors, translation_point)
# sun_vectors[:, 2] = (sun_vectors[:, 2])*-1

print(sun_vectors.shape)

(158, 3)


In [7]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(dt.tri_to_pv(context_mesh), color='#aaaaaa')

# add the sun locations, color orange
p.add_points(sun_points, color='#ffa500')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(85288.07383067516, 448664.0178023606, 1030.6752637571387),
 (84388.23267984093, 447764.17665152636, 130.83411292291257),
 (0.0, 0.0, 1.0)]

## 4. Compute Intersection of Sun Rays with Context Mesh

### 4.1. Preparing the List of Ray Directions and Origins

In [8]:
# constructing the sun direction from the sun vectors in a numpy array
sun_dirs = np.array(sun_vectors)
# exract the centroids of the envelope voxels
vox_cens = envelope_lattice.centroids
# next step we need to shoot in all of the sun directions from all of the voxels. To do so, we need to repeat the sun direction for the number of voxels to construct all ray directions (ray_dir). Also, we need to repeat the voxels for the amount of rays that it will receive. This results in a total amount of rays to be shooted (ray_src).  

ray_dir = np.tile(sun_dirs, [len(vox_cens),1])
ray_src = np.tile(vox_cens, [1, len(sun_dirs)]).reshape(-1, 3)


print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sun_dirs.shape)
print("number of rays to be shooted :",ray_src.shape)

number of voxels to shoot rays from : (287, 3)
number of rays per each voxel : (158, 3)
number of rays to be shooted : (45346, 3)


### 4.2. Computing the Intersection

In [9]:
# computing the intersections of rays with the context mesh from voxel centroid towards sun 
f_tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)

In [ ]:
sunray_id = pd.DataFrame(ray_id)
# save the sun access lattice to csv
lattice_dir = '../data/CSV_files/sun_hits.csv'
dt.save_to_clean_csv(sunray_id, lattice_dir)

## daylight


In [15]:
#Create a sphere to put points on that represent the sky 


sky_vectors = dt.sky_positions()
sky_points = np.add(sky_vectors, translation_point)

In [16]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(dt.tri_to_pv(context_mesh), color='#aaaaaa')

# add the sun locations, color orange
p.add_points(sky_points, color='#ffa500')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(85352.81071094499, 448782.9175240621, 1114.9455572142892),
 (84387.1151537307, 447817.22196684784, 149.25),
 (0.0, 0.0, 1.0)]

In [17]:
# constructing the sun direction from the sun vectors in a numpy array
sky_dirs = np.array(sky_vectors)

# next step we need to shoot in all of the sun directions from all of the voxels. To do so, we need to repeat the sun direction for the number of voxels to construct all ray directions (ray_dir). Also, we need to repeat the voxels for the amount of rays that it will receive. This results in a total amount of rays to be shooted (ray_src).  

sky_ray_dir = np.tile(sun_dirs, [len(vox_cens),1])
sky_ray_src = np.tile(vox_cens, [1, len(sun_dirs)]).reshape(-1, 3)


print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sky_dirs.shape)
print("number of rays to be shooted :",ray_src.shape)

number of voxels to shoot rays from : (287, 3)
number of rays per each voxel : (73, 3)
number of rays to be shooted : (45346, 3)


In [19]:
# computing the intersections of rays with the context mesh from voxel centroid towards sun 
f_tri_id_light, daylight_id = context_mesh.ray.intersects_id(ray_origins=sky_ray_src, ray_directions=sky_ray_dir, multiple_hits=False)

In [20]:
daylight_id = pd.DataFrame(daylight_id)
# save the sun access lattice to csv
lattice_dir = '../data/CSV_files/daylight_hits.csv'
dt.save_to_clean_csv(daylight_id, lattice_dir)